In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc("font", family = "gulim")
plt.rc("axes", unicode_minus = False)

fpath = "./01_Data_handling/dataset"

In [2]:
iris_df = sns.load_dataset("iris")
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
from sklearn.neighbors import KNeighborsClassifier # K-최근접 이웃(KNN) 알고리즘을 사용하여 분류 문제를 해결하기 위한 클래스
#KNN은 새로운 데이터를 예측할 때 가장 가까운 K개의 데이터를 참조하여 다수결로 결과를 결정하는 비모수적 분류 방법
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# 배우려고 일일히 했지만 앞으로는 이렇게 라이브러리를 쓰면 된다

In [4]:
X = iris_df.iloc[:, :-1]
y = iris_df.iloc[:, -1]

print(X.shape)
print(y.shape)

(150, 4)
(150,)


In [5]:
train_X, test_X, train_y, test_y, = train_test_split(
    X, y, test_size = 0.2
)

In [ ]:
ss = StandardScaler(with_mean=True, with_std=True) # 정규화한다면 sklearn에서는 다 이걸쓴다.
ss.fit(train_X)

train_scaled = ss.transform(train_X)
test_scaled = ss.transform(test_X)
print(ss.mean_)
print(ss.scale_)
print(ss.n_features_in_)

In [7]:
# train_scaled = ss.fit_transform(train_X) # 위처럼 안하고 이렇게 변환 하면 변환하고 나서 파라미터를 쓸수가 없다
# fit 트랜스폼은 어떤 때 쓰는가 

In [ ]:
# 타입 넘파이어레이
print(type(train_scaled))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_scaled, train_y)
print("Train acc = {:.4f}".format(knn.score(train_scaled, train_y)))
print("Test acc = {:.4f}".format(knn.score(test_scaled, test_y)))

In [ ]:
## Attribute
print("classes = ", knn.classes_)
# print("feature = ", knn.feature_names_in_)
print("metric = ", knn.effective_metric_) # : 평가지표
print("sample = ", knn.n_samples_fit_)
print("n neighbors = ", knn.n_neighbors)

In [ ]:
##
wine = pd.read_csv("https://bit.ly/wine-date")
print(wine.head())
print(wine.info())

In [ ]:
wine["class"] = wine["class"].astype("int32").astype("category")
wine.info()
wine.head()

In [ ]:
sns.boxplot(wine, x = "class", y = "sugar")
plt.ylim(0, 35)
plt.show()

sns.boxplot(wine, x = "class", y = "alcohol")
plt.ylim(5, 20)
plt.show()

sns.boxplot(wine, x = "class", y = "pH")
plt.ylim(0, 5)
plt.show()

In [ ]:
# wine.head()
X = wine.iloc[:, :-1]
y = wine.iloc[:, -1]

print(X.shape)
print(y.shape)

In [ ]:
## Split
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.2
)

print("train shape = ", train_X.shape)
print("test shape = ", test_X.shape)

In [ ]:
## Scaling
standScaler = StandardScaler()
standScaler.fit(train_X)

print(standScaler.mean_)
print(standScaler.scale_)

train_scaled = standScaler.transform(train_X)
test_scaled = standScaler.transform(test_X)

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(criterion="gini", max_depth=5, max_leaf_nodes=10) # 오버피팅?
# max_depth=5 더했더니 트레인이 떨어지고 테스트랑 비슷해짐 # max_leaf_nodes 마지막 leaf노드를 제어해서 오버피팅을 방지
dt.fit(train_scaled, train_y)
# 맞출때까지 끝까지 노드타고 비교해서 내려가니 트레인을 거의 다 맞춘다
print("Train acc = ", dt.score(train_scaled, train_y))
print("Test acc = ", dt.score(test_scaled, test_y))

In [ ]:
from sklearn.tree import plot_tree

plot_tree(dt, max_depth=3, filled=True) # filled
plt.show()

In [ ]:
print(dt.feature_importances_)

pd.DataFrame(dt.feature_importances_, index = train_X.columns)

In [25]:
## 교차검증 (Cross validation, cv)
from sklearn.model_selection import cross_validate, StratifiedKFold, GridSearchCV

In [26]:
cv_X_scaled = ss.fit_transform(X)

In [ ]:
print(y.value_counts())

In [28]:
splitter = StratifiedKFold(n_splits=5, shuffle=True) # stratigied가 비율을 반영해서 뽑는다
cv_result = cross_validate(dt, cv_X_scaled, y, cv = 5) # train_scaled, train_y, cv = 5
# 데이터 전체를 나누지않고 갖다 쓸 수 있...나..?
# cv는 사실상 학습에 모든데이터를 학습하고 test는 학습에 참여하지 않지만 
# cv한다고 할땐 굳이 train 과 test를 나눌필요가 없다?
# 

In [ ]:
print(cv_result["test_score"].mean()) # 데이터를 나누지 않더라도 그에 준하는 결과값을 cross_validation 하면 된다?

In [ ]:
## GridSearch

dt = DecisionTreeClassifier()

params = {
    "min_impurity_decrease": np.arange(0.001, 0.01, 0.0001),
    "max_depth": [5, 30, 1],
    "min_samples_split": np.arange(2, 100, 10)
}

splitter = StratifiedKFold(n_splits=5, shuffle=True)
# cv_result = cross_validate(dt, cv_X_scaled, y, cv = splitter)

grid_cv = GridSearchCV(dt,
                       param_grid= params,
                       cv = splitter)

grid_cv.fit(cv_X_scaled, y)
print(grid_cv.cv_results_)

In [ ]:
grid_cv.cv_results_.keys()

In [ ]:
print(grid_cv.best_params_)
print(np.mean(grid_cv.cv_results_["mean_test_score"]))
# 트리가 가장 설명하기 쉬워서 예를 들지만 모든 알고리즘이 gridsearch가 필요

In [33]:
## Random Forest # 스케일링 할 필요 없음음
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
print(y)
print(cv_X_scaled.shape)
print(y.shape)

In [ ]:
params = {
    "n_estimators": [50, 80, 100, 120],
    "max_depth": [3, 5, 7, 9, 12, 15]
}

splitter = StratifiedKFold(n_splits=5, shuffle=True)
grid_rf = GridSearchCV(rf, param_grid=params, cv = splitter)
grid_rf.fit(cv_X_scaled, y)

In [ ]:
print(grid_rf.cv_results_.keys())

In [ ]:
print(grid_rf.best_params_)
print(grid_rf.cv_results_["mean_test_score"])
print(grid_rf.best_score_)

In [ ]:
rf_best = RandomForestClassifier(n_estimators=120,
                                 max_depth=15)
rf_best.fit(cv_X_scaled, y)

In [ ]:
print(rf_best.feature_importances_)
print(rf_best.score(cv_X_scaled, y))

In [40]:
##
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier()

params = {
    "n_estimators": [50, 80, 100, 120],
    "max_depth": [3, 5, 7, 9, 12, 15]
}

splitter = StratifiedKFold(n_splits=5, shuffle=True)
cv_score = cross_validate(gb, cv_X_scaled, y, cv = splitter)

grid_gb = GridSearchCV(gb, param_grid=params, cv = splitter)

grid_gb.fit(cv_X_scaled, y)
# print(np.mean(cv_score["train_score"]))
# print(np.mean(cv_score["test_score"]))

# 랜텀포레스트는 뭘 찾으려하고 gb는 에러를 찾으려한다 보통 gb가 잘맞는다
# 부스팅계열은 안맞는 샘플을 맞추려고 더 노력해서 일반적으로 gb알고리즘이 더 잘맞을 수 밖에 없다

In [ ]:
print(grid_gb.best_params_)
print(grid_gb.cv_results_["mean_test_score"])
print(grid_gb.best_score_)

In [101]:
## DT, rf ,gb survived == 1 alive titanic 문제
# 트리는 스케일링? 안해도 된다.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree

In [ ]:

titanic = sns.load_dataset("titanic")
print(titanic.head())

In [ ]:
titanic = titanic.loc[:, :'fare']
titanic.dropna(subset=["age"], axis = 0, inplace=True)
titanic.shape


In [ ]:
# feature과 타겟변수 설정
X = titanic.loc[:, 'pclass':'fare']
y = titanic["survived"]

print("X shape = ",X.shape)
print("y shape = ",y.shape)
X.head()

In [ ]:
X = pd.get_dummies(titanic, columns = ["sex"], drop_first=True)
print(X.head())

In [ ]:
##dt 
dt = DecisionTreeClassifier()

params = {
    "max_depth": range(3, 20, 1),
    "max_leaf_nodes": range(3, 20, 1)
}

splitter = StratifiedKFold(n_splits= 5, shuffle=True)

gs_dt = GridSearchCV(dt, param_grid=params, cv = splitter)
gs_dt.fit(X, y)

In [ ]:
print("Best parameters = ", gs_dt.best_estimator_)
print("cv score = {:4f}".format(gs_dt.best_score_))

In [ ]:
## rf
rf = RandomForestClassifier()

params = {
    "n_estimators": range(90, 120, 1),
    "max_depth": range(3, 15, 1)
}

splitter = StratifiedKFold(n_splits=5, shuffle=True)
grid_rf = GridSearchCV(rf, param_grid=params, cv = splitter)
grid_rf.fit(X, y)

In [ ]:
print("Best parameters = ", grid_rf.best_estimator_)
print("cv score = {:4f}".format(grid_rf.best_score_))

In [ ]:
##gb # gb에 설명은 안했는데 꼭 해야하는 걸 하나 설명하고 끝낸다
b = GradientBoostingClassifier()

params = {
    "learning_rate": np.arange(0.1, 1), # 요건 반드시 해야한다. gb에만 있는 파라미터이다
    "max_depth": range(3, 5) # 트리계열에서 나오는 값이라 대충 비슷한 파라미터
} # 리그래션은 이런 파라미터가 없고 뭐가 그룹이 없어서 크로스 어쩌고를 해야한다. 트레이닝과 테스트도 나눠야하고 등?
# 값이 무지하게 많은게 아니라면 이값이 실제값과 거의 같다다

splitter = StratifiedKFold(n_splits=5, shuffle=True)

grid_gb = GridSearchCV(gb, param_grid=params, cv = splitter)
grid_gb.fit(X, y)

In [ ]:
print("Best parameters = ", grid_gb.best_estimator_)
print("cv score = {:4f}".format(grid_gb.best_score_))

In [ ]:
##
wine = pd.read_csv("https://bit.ly/wine-date")
wine.head()

In [ ]:
wine["class"] = wine["class"].astype("int32").astype("category")
wine.head()
wine.info()

In [128]:
X = wine.iloc[:, :-1]
y = wine["class"]

In [ ]:
## 반드시 스케일링을 하는게 좋다
standScaler = StandardScaler()
X_scaled = standScaler.fit_transform(X)
X_scaled[:10]


In [131]:
from sklearn.linear_model import LogisticRegression

In [133]:
#파라미터가 gridshearch할게 없다
lr = LogisticRegression(max_iter=100) # max_iter만 손댈게 있지마 이건 하이퍼 파라미터가 아니다

splitter = StratifiedKFold(n_splits=5, shuffle=True)
scores = cross_validate(lr, X_scaled, y, cv = splitter)

In [ ]:
print(scores["test_score"]) #아까 것? ? 이 더 좋지만 원리적으로 로지스틱도 되게 많이 찾음

In [ ]:
lr.fit(X_scaled, y)

In [ ]:
print("probability = \n", lr.predict_proba(X_scaled[:10]))
print("coefficient = \n", lr.coef_, lr.intercept_)
print("classes = \n", lr.classes_)
# 첫번째 P(x1)= 1/ 1+ e**-(1.79+0.53x1+1.65*x2 + -0.79*x3 #) 로오지스틱


In [ ]:
## DT, rf ,gb survived == 1 alive titanic 문제
'''
# train/test Split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)  # test_size 20%
print("train shape = ", train_X.shape)
print("test shape = ", test_X.shape)
# scaling
standScaler = StandardScaler() # 스케일링 클래스 객체 생성
standScaler.fit(train_X) # 훈련 데이터로 스케일링 객체 학습
# 스케일러의 평균과 스케일링 파라미터 확인
print(standScaler.mean_)
print(standScaler.scale_)'''

## dt
# 결정트리 모델 생성  (gini 기준, max_depth 5, max_leaf_nodes 10)
dt = DecisionTreeClassifier(criterion="gini", max_depth=5, max_leaf_nodes=10) # 오버피팅?
# max_depth=5 더했더니 트레인이 떨어지고 테스트랑 비슷해짐 # max_leaf_nodes 마지막 leaf노드를 제어해서 오버피팅을 방지
dt.fit(train_scaled, train_y) # 훈련 데이터로 모델 학습
# 훈련 데이터와 테스트 데이터에서의 정확도 확인 # 맞출때까지 끝까지 노드타고 비교해서 내려가니 트레인을 거의 다 맞춘다
print("Train acc = ", dt.score(train_scaled, train_y))
print("Test acc = ", dt.score(test_scaled, test_y))
# 의사결정 트리 시각화
plot_tree(dt, max_depth=3, filled=True) # filled
plt.show()

# print(dt.feature_importances_)
# pd.DataFrame(dt.feature_importances_, index = train_X.columns)